# Transition matrices and Credit Metrics


In [1]:
#Import packages
import pandas as pd
import numpy as np
import scipy 
from scipy.stats import norm


from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm


In [2]:
# Read Excel file with transition probabilities
table = pd.read_excel('MigrationTable.xlsx')
print(table)






  Rating    AAA     AA      A    BBB     BB      B    CCC       D
0    AAA  90.81   8.33   0.68   0.06   0.12   0.00   0.00    0.00
1     AA   0.70  90.65   7.79   0.64   0.06   0.14   0.02    0.00
2      A   0.09   2.27  91.05   5.52   0.74   0.26   0.01    0.06
3    BBB   0.02   0.33   5.95  86.93   5.30   1.17   0.12    0.18
4     BB   0.03   0.14   0.67   7.73  80.53   8.84   1.00    1.06
5      B   0.00   0.11   0.24   0.43   6.48  83.46   4.07    5.20
6    CCC   0.22   0.00   0.22   1.30   2.38  11.24  64.86   19.79
7      D   0.00   0.00   0.00   0.00   0.00   0.00   0.00  100.00


## Transition matrices - Swiching time horizon
Background is Hull 19.1 Rating Transition matrices

In [3]:
one_year_tm = table.iloc[:,1:].to_numpy()/100
np.set_printoptions(precision=4,suppress=True) # Print using non scientific notation
print("This is the one year transition matrix")
print(one_year_tm)

five_year_tm = np.linalg.matrix_power(one_year_tm,5)
print("\n This is the five year transition matrix")
print(five_year_tm)

one_month_tm = scipy.linalg.fractional_matrix_power(one_year_tm,1/12)
print("\n This is the one month transition matrix")
print(one_month_tm) # Appendix J in Hull (- sign is due to some numerical approximation error)

five_thousand_year_tm = np.linalg.matrix_power(one_year_tm,50000)
print("\n This is the five thousand year transition matrix")
print(five_thousand_year_tm) # % In the long-run we are all dead

This is the one year transition matrix
[[0.9081 0.0833 0.0068 0.0006 0.0012 0.     0.     0.    ]
 [0.007  0.9065 0.0779 0.0064 0.0006 0.0014 0.0002 0.    ]
 [0.0009 0.0227 0.9105 0.0552 0.0074 0.0026 0.0001 0.0006]
 [0.0002 0.0033 0.0595 0.8693 0.053  0.0117 0.0012 0.0018]
 [0.0003 0.0014 0.0067 0.0773 0.8053 0.0884 0.01   0.0106]
 [0.     0.0011 0.0024 0.0043 0.0648 0.8346 0.0407 0.052 ]
 [0.0022 0.     0.0022 0.013  0.0238 0.1124 0.6486 0.1979]
 [0.     0.     0.     0.     0.     0.     0.     1.    ]]

 This is the five year transition matrix
[[0.622  0.2851 0.073  0.0121 0.0051 0.002  0.0003 0.0004]
 [0.0244 0.6302 0.2722 0.0523 0.0103 0.0076 0.001  0.0018]
 [0.0044 0.0804 0.6641 0.1823 0.0416 0.0187 0.0021 0.0064]
 [0.0014 0.0211 0.1957 0.5463 0.1435 0.0625 0.0084 0.021 ]
 [0.0013 0.0083 0.0516 0.2038 0.3968 0.2194 0.0322 0.0867]
 [0.0007 0.0047 0.0153 0.0459 0.159  0.4609 0.0691 0.244 ]
 [0.0044 0.0027 0.0126 0.0391 0.0728 0.1933 0.1334 0.5417]
 [0.     0.     0.     0.     0. 

## Pieces of Credit metrics
This code gives you some pieces you need for Computer exercise 4 in which you will implement the Credit Metrics model with Monte Carlo simulation
### Quantiles from Transition matrix
How to replicate the quantiles calculated at VL16 slide 5

In [4]:
cumulative_probs_BBB = np.cumsum(np.flip(one_year_tm[3,:]))
print(cumulative_probs_BBB)

[0.0018 0.003  0.0147 0.0677 0.937  0.9965 0.9998 1.    ]


So 0.0018 is prob of moving from BBB to D, 0.0030 is prob of moving from BBB to CCC or lower,... 1 is prob of moving from BBB to AAA or lower

In [5]:
quantiles_BBB = norm.ppf(cumulative_probs_BBB[:-1]) # Skipping the upper bound for AAA since it is infinity
print(quantiles_BBB)

[-2.9112 -2.7478 -2.1781 -1.4931  1.5301  2.6968  3.5401]


Interpretation is that a number <-2.91 means a bond ends up in D. >3.54 in AAA, between -1.49 and 1.53 stays in BBB etc.
The quantiles gives us a distribution based on the transition matrix to simulate credit risk. If you draw

In [6]:
np.random.seed(1) # Set random seed for reproducability
x1 = np.random.normal(0, 1, 1) # One standard normal variable
print(x1)

[1.6243]


1.6243 means the bond is upgraded to A

## Generating correlated random numbers
It is generally hard (involving copulas) to generate correlation between two random variables X1 and X2. Easier when X1 and X2 are Gaussian as in credit metrics.

In [7]:
np.random.seed(1) # Set random seed for reproducability
x1 = np.random.normal(0,1,100000) # 100,000 random numbers
x2 = np.random.normal(0,1,100000) # Another 100,000 random numbers


x1 and x2 are independent (for Gussian random variables uncorrelated implies independent) by construction but we check just in case

In [8]:
print(scipy.stats.pearsonr(x1, x2)) # Correlation and p-stat for null of correlation=0


PearsonRResult(statistic=-0.005551580831574797, pvalue=0.0791643214605501)


## Don't do this

In [9]:
np.random.seed(1) # Set random seed for reproducability
x1 = np.random.normal(0,1,100000) # 100,000 random numbers
np.random.seed(1) # Set random seed for reproducability
x2 = np.random.normal(0,1,100000) # Another 100,000 random numbers
print(scipy.stats.pearsonr(x1, x2)) # Correlation and p-stat for null of correlation=0

PearsonRResult(statistic=0.9999999999999999, pvalue=0.0)


Perfect correlation since you generate the same random numbers twice (computer generated "random" numbers are deterministic). 


## Make q1 and q2 correlated with R=0.35

In [10]:
x1 = np.random.normal(0,1,100000) # 100,000 random numbers
x2 = np.random.normal(0,1,100000) # Another 100,000 random numbers
R = 0.35
q1 = x1
q2 = R*x1+np.sqrt(1-R**2)*x2 # Cholesky decomposition VL16 slides 13-15
print(scipy.stats.pearsonr(q1, q2)) # Correlation and p-stat for null of correlation=0

PearsonRResult(statistic=0.3470917840328057, pvalue=0.0)


Another way to do this is with the Multivariate Normal distribution, with this method we can easily increase the number of bonds and have different correlations between different bonds

In [11]:
mu = [0,0]
cov = [[1, R], [R, 1]] # 2x2 Covariance matrix
x, y = np.random.multivariate_normal(mu, cov, 100000).T # .T puts results in two variables

Covariance equals correlation since we have standardized varibles (standardized log asset value in Credit Metrics is N(0,1))

In [12]:
print(scipy.stats.pearsonr(x, y)) # Correlation and p-stat for null of correlation=0

PearsonRResult(statistic=0.34770129745454337, pvalue=0.0)
